In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('max_colwidth', 240)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import re
import os
import operator
import sys
print(os.listdir("../input"))

import matplotlib.pyplot as plt

# Any results you write to the current directory are saved as output.

In [ ]:
MAX_NB_WORDS = 120000
GLOVE_DIR = "../input/embeddings/glove.840B.300d"
EMBEDDING_DIM = 300
MAX_LEN = 75

## Read Data

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
df = pd.concat([train.drop('target',axis=1),test])
# Comment after debugging, before committing:
#df = df.sample(150000)

## Clean Data

In [ ]:
#X = traindf["question_text"].fillna("_##_").values
#y = traindf["target"].values

## Feature Selection

In [ ]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.840B.300d.txt'))
for line in f:
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [ ]:
def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

In [ ]:
vocab = build_vocab(df["question_text"])

In [ ]:
print("Glove : ")
oov_glove = check_coverage(vocab, embeddings_index)

In [ ]:
oov_glove[:10]

In [ ]:
contraction_mapping = {"ain't": "is not", 
                       "aren't": "are not",
                       "can't": "cannot", 
                       "'cause": "because", 
                       "could've": "could have",
                       "couldn't": "could not", 
                       "didn't": "did not",  
                       "doesn't": "does not", 
                       "don't": "do not", 
                       "hadn't": "had not", 
                       "hasn't": "has not", 
                       "haven't": "have not", 
                       "he'd": "he would",
                       "he'll": "he will", 
                       "he's": "he is", 
                       "how'd": "how did", 
                       "how'll": "how will", 
                       "how's": "how is",  
                       "I'd": "I would", 
                       "I'll": "I will", 
                       "I'm": "I am", 
                       "I've": "I have", 
                       "i'd": "i would", 
                       "i'd've": "i would have", 
                       "i'll": "i will",  
                       "i'm": "i am", 
                       "i've": "i have", 
                       "isn't": "is not", 
                       "it'd": "it would", 
                       "it'll": "it will", 
                       "it's": "it is", 
                       "let's": "let us", 
                       "might've": "might have",
                       "must've": "must have", 
                       "needn't": "need not", 
                       "o'clock": "of the clock", 
                       "shan't": "shall not", 
                       "she'd": "she would",
                       "she'll": "she will", 
                       "she's": "she is", 
                       "should've": "should have", 
                       "shouldn't": "should not", 
                       "that'd": "that would", 
                       "there's": "there is", 
                       "here's": "here is",
                       "they'd": "they would",
                       "they'll": "they will", 
                       "they're": "they are", 
                       "they've": "they have", 
                       "wasn't": "was not", 
                       "we'd": "we would", 
                       "we'll": "we will", 
                       "we're": "we are", 
                       "we've": "we have", 
                       "weren't": "were not", 
                       "what'll": "what will", 
                       "what're": "what are",  
                       "what's": "what is", 
                       "what've": "what have", 
                       "when's": "when is", 
                       "when've": "when have", 
                       "where'd": "where did", 
                       "where's": "where is", 
                       "where've": "where have", 
                       "who'll": "who will", 
                       "who's": "who is", 
                       "who've": "who have", 
                       "why's": "why is", 
                       "won't": "will not", 
                       "would've": "would have", 
                       "wouldn't": "would not", 
                       "y'all": "you all", 
                       "you'd": "you would", 
                       "you'd've": "you would have", 
                       "you'll": "you will", 
                       "you'll've": "you will have", 
                       "you're": "you are", 
                       "you've": "you have" }


In [ ]:
def known_contractions(embed):
    known = []
    for contract in contraction_mapping:
        if contract in embed:
            known.append(contract)
    return known

In [ ]:
print("- Known Contractions -")
print("   Glove :")
print(known_contractions(embeddings_index))

In [ ]:
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [ ]:
df['treated_question'] = df['question_text'].apply(lambda x: clean_contractions(x, contraction_mapping))

In [ ]:
vocab = build_vocab(df['treated_question'])
print("Glove : ")
oov_glove = check_coverage(vocab, embeddings_index)

In [ ]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [ ]:
def unknown_punct(embed, punct):
    unknown = ''
    for p in punct:
        if p not in embed:
            unknown += p
            unknown += ' '
    return unknown

In [ ]:
print("Glove :")
print(unknown_punct(embeddings_index, punct))

In [ ]:
punct_mapping = {"‘": "'", 
                 "₹": "e", 
                 "´": "'", 
                 "°": "", 
                 "€": "e", 
                 "™": "tm", 
                 "√": " sqrt ", 
                 "×": "x", 
                 "²": "2", 
                 "—": "-", 
                 "–": "-", 
                 "’": "'",
                 "_": "-", 
                 "`": "'", 
                 '“': '"', 
                 '”': '"', 
                 '“': '"', 
                 "£": "e", 
                 '∞': 'infinity', 
                 'θ': 'theta', 
                 '÷': '/', 
                 'α': 'alpha', 
                 '•': '.', 
                 'à': 'a', 
                 '−': '-', 
                 'β': 'beta', 
                 '∅': '', 
                 '³': '3', 
                 'π': 'pi'}

In [ ]:
def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  # Other special characters that I have to deal with in last
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

In [ ]:
df['treated_question'] = df['treated_question'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))

In [ ]:
vocab = build_vocab(df['treated_question'])
print("Glove : ")
oov_glove = check_coverage(vocab, embeddings_index)

In [ ]:
oov_glove[:100]

In [ ]:
mispell_dict = {'Quorans': 'Quora',
                'cryptocurrencies': 'cryptocurrency',
                'Redmi' :'cellphone',
                'OnePlus' :'cellphone',
                'Blockchain':'blockchain',
                'Pokémon':'Pokemon',
                'ethereum':'Ethereum',
                'Qoura':'Quora',
                'fiancé':'fiance',
                'Litecoin': 'cryptocurrency',
                'altcoin': 'cryptocurrency',
                'Cryptocurrency': 'cryptocurrency',
                'altcoins' : 'cryptocurrency',
                'litecoin' : 'cryptocurrency',
                'colour': 'color',
                'centre': 'center',
                'favourite': 'favorite',
                'travelling': 'traveling',
                'counselling': 'counseling',
                'theatre': 'theater',
                'cancelled': 'canceled',
                'labour': 'labor',
                'organisation': 'organization',
                'wwii': 'world war 2',
                'citicise': 'criticize',
                'youtu ': 'youtube ',
                'Qoura': 'Quora',
                'sallary': 'salary',
                'Whta': 'What',
                'narcisist': 'narcissist',
                'howdo': 'how do',
                'whatare': 'what are',
                'howcan': 'how can',
                'howmuch': 'how much',
                'howmany': 'how many',
                'whydo': 'why do',
                'doI': 'do I',
                'theBest': 'the best',
                'howdoes': 'how does',
                'mastrubation': 'masturbation',
                'mastrubate': 'masturbate',
                "mastrubating": 'masturbating',
                'pennis': 'penis',
                'Etherium': 'Ethereum',
                'narcissit': 'narcissist',
                'bigdata': 'big data', 
                '2k17': '2017', 
                '2k18': '2018', 
                'qouta': 'quota', 
                'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', 
                "whst": 'what', 'watsapp': 'whatsapp', 
                'demonitisation': 'demonetization', 
                'demonitization': 'demonetization', 
                'demonetisation': 'demonetization'}

In [ ]:
def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

In [ ]:
df['treated_question'] = df['treated_question'].apply(lambda x: correct_spelling(x, mispell_dict))

In [ ]:
vocab = build_vocab(df['treated_question'])
print("Glove : ")
oov_glove = check_coverage(vocab, embeddings_index)

In [ ]:
y = pd.read_csv('../input/train.csv')['target']
X = df[:len(y)]['treated_question']

In [ ]:
%%time
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
# tokenizer: words --> word indices

texts, labels = X, y

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(list(X))
sequences = tokenizer.texts_to_sequences(X)

sequences = pad_sequences(sequences, maxlen=MAX_LEN)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
# Example sequence
sequences[0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(sequences, labels, test_size=0.2)
y_train_bin = to_categorical(y_train)
y_val_bin = to_categorical(y_val)

In [ ]:
print('Vector for \'apple\' =')
print(str(embeddings_index['apple'][0:10]) + '... plus 290 more features')

In [ ]:
%%time
# embedding_matrix: word index --> word vector
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
ai = word_index['apple']
print('\'apple\' is word number: ', ai)
print('The vector for word number ', ai, ' is:')
print(str(embedding_matrix[ai][0:10]) + '... plus 290 more features')

In [ ]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_LEN,
                            trainable=False)

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Bidirectional, CuDNNGRU, GlobalMaxPool1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.models import Model
from keras import regularizers

model = Sequential()
#model.add(Input(shape=(MAX_LEN,), dtype='int32'))
model.add(embedding_layer)
model.add(Bidirectional(CuDNNGRU(64, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation="sigmoid"))
model.add(Dropout(0.3))
model.add(Dense(32, activation="sigmoid"))
model.add(Dropout(0.3))
model.add(Dense(2, activation="softmax"))

#model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
from keras import callbacks
from sklearn import metrics
class cbmetrics(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        scores = self.model.predict(X_val, batch_size=1024, verbose=1)[:,1]
        pr, re, th = metrics.precision_recall_curve(y_val, scores)
        pr, re, th = pr[:-2], re[:-2], th[:-1]
        fs = 2*np.divide(np.multiply(pr, re), np.add(pr, re))
        i = np.argmax(fs)
        self.val_f1s.append(np.max(fs))
        self.val_recalls.append(re[i])
        self.val_precisions.append(pr[i])
        print("f score: ", np.max(fs))
        
f1 = cbmetrics()

In [ ]:
history = model.fit(X_train, y_train_bin, validation_data=(X_val, y_val_bin),
          epochs=3, batch_size=256, verbose=1, callbacks=[f1])

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
val_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, val_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
scores = model.predict(X_val, batch_size=1024, verbose=1)[:,1]

In [ ]:
from sklearn import metrics
pr, re, th = metrics.precision_recall_curve(y_val, scores)
pr, re, th = pr[:-2], re[:-2], th[:-1]
fs = 2*np.divide(np.multiply(pr, re), np.add(pr, re))

plt.figure(figsize=(12,8))
plt.plot(th, pr, label = 'precision')
plt.plot(th, re, label = 'recall')
plt.plot(th, fs, label = 'f score')
plt.xlabel('Threshold')
plt.legend()

In [ ]:
opt_thr = th[np.argmax(fs)]
print(opt_thr, np.max(fs))

In [ ]:
# Next 2 cells are from https://www.kaggle.com/frankkloster/time-to-put-your-gloves-on
df_test = df[len(y):]
x_test = df_test["treated_question"]
x_test = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(x_test, maxlen=MAX_LEN)

In [ ]:
y_test = model.predict([x_test], batch_size=1024, verbose=1)[:,1]
y_test = (y_test > opt_thr).astype(int)
df_test = pd.DataFrame({"qid": df_test["qid"].values})
df_test['prediction'] = y_test
df_test.to_csv("submission.csv", index=False)